In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
import numpy as np

In [2]:
import warnings
warnings.filterwarnings('ignore')

In [3]:
data=pd.read_csv('Database.csv',index_col=None)

In [4]:
Label=data.iloc[:,-22:-3]
for i in range(Label.shape[0]):
    for j in range(Label.shape[1]):
        if Label.iloc[i,j]!=0:
            Label.iloc[i,j]=1
Label.to_csv('Label.csv',index=False)

In [5]:
x=data.iloc[:,4:12]
x_encode=pd.get_dummies(x,columns=x.columns)
x_encode.to_csv('x_encode.csv',encoding='utf-8_sig')
x_encode.shape


(99, 81)

In [6]:
X_train,X_test,Y_train,Y_test=train_test_split(x_encode,Label,test_size=0.25,shuffle=True)

In [7]:
from sklearn.pipeline import Pipeline
from sklearn.multioutput import MultiOutputClassifier
##from xgboost import XGBClassifier
##from sklearn.neural_network import MLPClassifier
from sklearn.ensemble import RandomForestClassifier
Multi_Classifier=MultiOutputClassifier(RandomForestClassifier())
clf_classification=Pipeline([('Classifier',Multi_Classifier)])
clf_classification.fit(X_train,Y_train)
y_pred=pd.DataFrame(Multi_Classifier.predict(X_test))
y_pred.columns=pd.DataFrame(Label).columns
y_pred.to_csv('D:/OneDrive/SRTP/output_pred.csv',encoding='utf-8_sig')
Y_test.to_csv('D:/OneDrive/SRTP/output_test.csv',encoding='utf-8_sig')

In [8]:
from joblib import dump
dump(clf_classification, 'predictnucleoside.joblib')

['predictnucleoside.joblib']

In [9]:
from sklearn.metrics import classification_report
print(classification_report(Y_test,y_pred))
from sklearn.metrics import hamming_loss
print(hamming_loss(Y_test,y_pred))

              precision    recall  f1-score   support

           0       0.00      0.00      0.00         1
           1       0.73      0.69      0.71        16
           2       0.56      0.71      0.63         7
           3       0.82      0.75      0.78        12
           4       0.33      0.17      0.22         6
           5       1.00      0.33      0.50         3
           6       0.95      0.86      0.90        22
           7       0.00      0.00      0.00         1
           8       0.00      0.00      0.00         3
           9       0.50      0.78      0.61         9
          10       0.75      0.40      0.52        15
          11       0.00      0.00      0.00         2
          12       0.73      0.89      0.80        18
          13       0.00      0.00      0.00         2
          14       0.75      0.75      0.75         8
          15       0.78      1.00      0.88        18
          16       0.00      0.00      0.00         1
          17       1.00    